In [ ]:
from operator import methodcaller

import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

sns.set_theme()


In [ ]:
df = pd.read_csv(
    "out/formatted/formatted.csv", index_col="timestamp", parse_dates=["timestamp"]
)
df.columns.name = "studio"
df.columns = df.columns.map(methodcaller("title"))


In [ ]:
import numpy as np


ignore = [
    ("2022-11-15 09:26:00+0100", "Eppendorf"),
    ("2022-11-15 18:19:00+0100", "Eppendorf"),
    ("2022-11-16 14:40:00+0100", "Eppendorf"),
    ("2022-11-18 04:16:00+0100", "Eppendorf"),
    ("2022-11-18 20:16:00+0100", "Eppendorf"),
    ("2022-11-20 05:25:00+0100", "Eppendorf"),
    ("2022-11-21 00:19:00+0100", "Eppendorf")
]

for i, c in ignore:
    df.loc[i, c] = np.nan


In [ ]:
df.stack().to_frame("occupancy").query("occupancy == 100")

In [ ]:
plt.figure(figsize=(10, 5), dpi=200)
sns.lineplot(df)


In [ ]:
plt.figure(figsize=(10, 5), dpi=200)
sns.lineplot(df.resample("H").mean().interpolate())


In [ ]:
hour_min, hour_max = 8, 22
plt.figure(figsize=(10, 5), dpi=200)
sns.lineplot(
    df[df.index.hour.to_series(index=df.index).between(hour_min, hour_max)]
    .resample("D")
    .mean()
    .interpolate()
)
plt.title(f"Mean occupancy between {hour_min} and {hour_max}")


In [ ]:
plt.figure(figsize=(10, 5), dpi=200)
sns.lineplot(df.resample("D").max().interpolate())
plt.title(f"Max occupancy")


In [ ]:
hour = 17
plt.figure(figsize=(10, 5), dpi=200)
sns.lineplot(df.query("index.dt.hour >= @hour").resample("D").max().interpolate())
plt.title(f"Max occupancy after {hour}")


In [ ]:
df_resampled = df.resample("H").mean().interpolate()


In [ ]:
def seasonality_plot(
    df: pd.DataFrame, title: str
) -> None:
    plt.figure(figsize=(10, 5), dpi=200)
    df["hour"] = df.index.hour
    df["day"] = df.index.floor("d")
    plt.title(title)
    sns.lineplot(data=df, x="hour", y="occupancy", hue="day", palette="viridis")
    plt.axvline(17, color="gray", linestyle="--")


for c in df_resampled:
    df_plot = df_resampled[c].to_frame().rename(columns={c: "occupancy"})
    seasonality_plot(df_plot, c)
    plt.show()


In [ ]:
for c in df_resampled:
    df_plot = df_resampled[c].to_frame().rename(columns={c: "occupancy"})
    seasonality_plot(df_plot[df_plot.index.day_of_week < 5].copy(), f"{c} Arbeitstag")
    plt.show()


In [ ]:
for c in df_resampled:
    df_plot = df_resampled[c].to_frame().rename(columns={c: "occupancy"})
    seasonality_plot(df_plot[df_plot.index.day_of_week >= 5].copy(), f"{c} Wochenende")
    plt.show()
